## Cytokine analysis app

In [1]:
################### Importing packages ###################

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pylab import *
import seaborn as sns
from statannot import add_stat_annotation
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import math

################### Function that downloads data and calls widgets ###################

# pipeline for extracting data matrix containing cytokine information we need
def cytokine_controller():
    
    ################### Pre-process excel file ###################

    # loading in entire excel file that comes out of multi-plex machine -- we only care about the sheet "Conc in Range"
    cytokine_raw_df = pd.ExcelFile('../../../data/cytokine_data/Human-10plex.xlsx') # 
    cytokine_raw_concs_df = pd.read_excel(cytokine_raw_df, 'Conc in Range', header=7) # skip first 7 rows -- just file/machine info

    # Creating list cotaining elements in first column -- this will help us parse out information we care about
    list_of_well_types = list(cytokine_raw_concs_df.iloc[:,0])

    # find indices of "type" -- again, this helps us parse the data frame
    idx_of_type =  [i for i, x in enumerate(list_of_well_types) if x == "Type"]
    start_df_idx = idx_of_type[1]

    # index dataframe such that it starts 4-rows after we see type (those rows just contain "background")
    # and ends 5 rows before the final row (which just contain labeling information)
    cytokine_concs_df = cytokine_raw_concs_df.iloc[start_df_idx+4:-5:,:]

    # renamining columns and indices
    cytokine_concs_df = cytokine_concs_df.rename(columns = {"Unnamed: 0":"Type", "Unnamed: 1": "Well",
                                                            "Unnamed: 2": "Description" })
    # only keep columns of interest (cytokines + descriptors )
    cytokine_concs_df = cytokine_concs_df.rename(index= dict(zip(list(cytokine_concs_df.index),list(cytokine_concs_df.loc[:,"Well"])))).drop("Well", axis=1)

    # plate map -- hard-coded
    plate_map_dict = {
        "NHB control N3": ['A4', 'A5', 'A6'],
        "NHB 0.1 MOI N3": ['B4', 'B5', 'B6'],
        "NHB 0.05 MOI N2": ['C4', 'C5', 'C6'],
        "NHB 0.05 MOI N3": ['D4', 'D5', 'D6'],
        "NHB 0.01 MOI N2" : ['E4', 'E5', 'E6'],
        "NHB 0.01 MOI N3": ['F4', 'F5', 'F6'],
        "COPD control N3": ['A7', 'A8', 'A9'],
        "COPD 0.1 MOI N3": ['B7', 'B8', 'B9'],
        "COPD 0.05 MOI N2": ['C7', 'C8', 'C9'],
        "COPD 0.05 MOI N3": ['D7', 'D8', 'D9'],
        "COPD 0.01 MOI N2" : ['E7', 'E8', 'E9'],
        "COPD 0.01 MOI N3": ['F7', 'F8', 'F9']
    }

    # adding more parseable meadata to dataframe
    for condition, values in plate_map_dict.items():
        condition_all_info = condition.split(' ')

        if 'control' not in condition_all_info:
            cytokine_concs_df.loc[values[0], 'Condition'] = condition
            cytokine_concs_df.loc[values[0], 'MOI'] = condition_all_info[1]
            cytokine_concs_df.loc[values[0], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[0], 'disease_status'] = condition_all_info[0]

            cytokine_concs_df.loc[values[1], 'Condition'] = condition
            cytokine_concs_df.loc[values[1], 'MOI'] = condition_all_info[1]
            cytokine_concs_df.loc[values[1], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[1], 'disease_status'] = condition_all_info[0]

            cytokine_concs_df.loc[values[2], 'Condition'] = condition
            cytokine_concs_df.loc[values[2], 'MOI'] = condition_all_info[1]
            cytokine_concs_df.loc[values[2], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[2], 'disease_status'] = condition_all_info[0]
        else:
            cytokine_concs_df.loc[values[0], 'Condition'] = condition
            cytokine_concs_df.loc[values[0], 'MOI'] = 'control'
            cytokine_concs_df.loc[values[0], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[0], 'disease_status'] = condition_all_info[0]

            cytokine_concs_df.loc[values[1], 'Condition'] = condition
            cytokine_concs_df.loc[values[1], 'MOI'] = 'control'
            cytokine_concs_df.loc[values[1], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[1], 'disease_status'] = condition_all_info[0]

            cytokine_concs_df.loc[values[2], 'Condition'] = condition
            cytokine_concs_df.loc[values[2], 'MOI'] = 'control'
            cytokine_concs_df.loc[values[2], 'transwell'] = condition_all_info[-1]
            cytokine_concs_df.loc[values[2], 'disease_status'] = condition_all_info[0]

    # removing OOR values and replacing with NaN
    cytokine_concs_df = cytokine_concs_df.replace(np.nan, 'Standard', regex=True) 
    cytokine_concs_df = cytokine_concs_df.replace('OOR >', np.nan, regex=True)
    cytokine_concs_df = cytokine_concs_df.replace('OOR <', np.nan, regex=True)

    # renaming cytokines so we keep their protein names and remove the parentheses
    cytokines = [cyto.split('(')[0][:-1] for cyto in cytokine_concs_df.columns[2:-4]]
    cytokine_name_replace_dict = dict(zip(list(cytokine_concs_df.columns[2:-4]), cytokines))
    cytokine_concs_df = cytokine_concs_df.rename(columns = cytokine_name_replace_dict)
    plate_map_list = list(plate_map_dict.keys())
    plate_map_list_final = list(set([cond[:-3] for cond in plate_map_list]))
    potential_conds_to_compare = ["None"] + sorted(set(plate_map_list_final), reverse=True) # , key=plate_map_list_final.index
    
    # creating widgets for multiple user options:
    # which cytokine to plot
    cytokine_to_plot = widgets.Dropdown(options = cytokines,
                          value = cytokines[0],
                          description="Cytokine:")

    # which transwell to plot
    transwell_to_plot = widgets.Dropdown(options =['N2', 'N3'],
                          value = 'N3',
                          description="N3 or N2?:")
    statistical_test_to_plot = widgets.Dropdown(options =['Mann-Whitney', 't-test_ind'],
                          value = 't-test_ind',
                          description="Stat test:")

    # how to display statistical test
    p_val_to_plot = widgets.Dropdown(options =['star', 'simple'],
                          value = 'simple',
                          description="pVal display:")
   
    # show mean values
    showMeans_to_plot = widgets.Dropdown(options =[False, True],
                          value = False,
                          description="Show means:")
    # condition 1 you want to compare
    condition_1_to_compare = widgets.Dropdown(options = potential_conds_to_compare,
                            value = potential_conds_to_compare[0],
                            description="Condition 1 :")
    # condition 2 you want to compare
    condition_2_to_compare = widgets.Dropdown(options = potential_conds_to_compare,
                            value = potential_conds_to_compare[0],
                            description="Condition 2 :")
    # save the plot
    save_plot = widgets.Dropdown(options =[False, True],
                          value = False,
                          description="save plot:")

    return(widgets.interactive(plot_cytokine,
                       cytokine_concs_df=fixed(cytokine_concs_df),
                        cytokine_name=cytokine_to_plot, 
                        transwell=transwell_to_plot,
                       statistical_test = statistical_test_to_plot,
                       print_p_val = p_val_to_plot,
                       means=showMeans_to_plot,
                        condition_to_compare_1=condition_1_to_compare,
                        condition_to_compare_2=condition_2_to_compare,   
                       save_plot =save_plot))       

################### function for plotting data ###################

# function used in interactive widget
def plot_cytokine(cytokine_concs_df, cytokine_name, transwell, 
                  statistical_test, print_p_val, means, condition_to_compare_1,
                  condition_to_compare_2, save_plot):
    
    # create figure object
    fig, ax = plt.subplots(figsize=(10,8), dpi=400)

    # dislay as stars or actual p-value
    if print_p_val == 'star':
        print_p_val = "star" # thresholds: [[1e-4, "****"], [1e-3, "***"], [1e-2, "**"], [0.05, "*"], [1, "ns"]]
    else:
        print_p_val = "simple"

    # choose to plot N3 or N2 wells
    if transwell == "N3":
        conditions_order = ["control", "0.01", '0.05', '0.1']
    else:
        conditions_order = ["0.01", '0.05']
    
    # color schemes
    light_pink = '#FFC9EC' 
    light_brown = '#E5B699'
    dark_brown = '#B25116'
    dark_pink = '#FB84D1'

    # pallete for boxplot and stripplot
    face_pal = {'NHB': light_brown, 'COPD': light_pink}
    pal = {'NHB': dark_brown, 'COPD': dark_pink}
    
    # order to plot results
    hue_order = ['NHB', 'COPD']

    # plot properties
    boxprops = {'edgecolor': 'k', 'linewidth': 2}
    lineprops = {'color': 'k', 'linewidth': 2}
    boxplot_kwargs = {'boxprops': boxprops, 'medianprops': lineprops,
                      'whiskerprops': lineprops, 'capprops': lineprops,
                      'width': 0.75, 'palette': face_pal,
                      'hue_order': hue_order}
    stripplot_kwargs = {'linewidth': 0.6, 'size': 6, 'alpha': 0.9,
                        'palette': pal, 'hue_order': hue_order}
    
    # removing standards from dataframe to simplify plotting 
    cytokine_concs_remove_standards_df = cytokine_concs_df[cytokine_concs_df['Condition'] != 'Standard']
    
    # choosing transwell -- argument for plotting
    cytokine_concs_specify_transwell_df = cytokine_concs_remove_standards_df[cytokine_concs_remove_standards_df['transwell'] == transwell]
    cytokine_name = cytokine_name
    
    # Plot, using all the same parameters as above
    sns.boxplot(x='MOI', y=cytokine_name, hue='disease_status', data=cytokine_concs_specify_transwell_df, ax=ax,
        fliersize=0, showmeans=means, order=conditions_order,**boxplot_kwargs)
    sns.stripplot(x='MOI', y=cytokine_name, hue='disease_status', data=cytokine_concs_specify_transwell_df, ax=ax,
        dodge=True, jitter=0.2,order=conditions_order, **stripplot_kwargs)

    # removing double legend 
    handles, labels = ax.get_legend_handles_labels()
    lgd = ax.legend(handles[0:2], labels[0:2],
                   loc='upper left',
                   fontsize='large',
                   handletextpad=0.5)
    lgd.legendHandles[0]._sizes = [40]
    lgd.legendHandles[1]._sizes = [40]
    plt.xlabel('MOI')
    plt.ylabel(cytokine_name + " (pg/$\mu$L)")
    
    # only perform statistical tests if there is greater than one value for cytokine
    
    # these variables help with choosign which values to test
    conditions_all = list(cytokine_concs_specify_transwell_df["Condition"])
    unique_conditions_list = list(unique([' '.join(cond.split(' ')[1:]) for cond in conditions_all]))
    conditions_count_dict = dict(zip(unique_conditions_list, [[0,0]] * len(unique_conditions_list)))

    # see which conditions DON'T have an NAN value
    for conc, cond in cytokine_concs_specify_transwell_df[[cytokine_name, 'Condition']].itertuples(index=False):
        cond_parse = cond.split(' ')

        patient_status = cond_parse[0]
        full_condition = ' '.join(cond_parse[1:])
        if math.isnan(conc):
            continue
        else:
            if patient_status == "NHB":
                conditions_count_dict[full_condition] =[conditions_count_dict[full_condition][0]+1, conditions_count_dict[full_condition][1]]
            elif patient_status == "COPD":
                conditions_count_dict[full_condition] =[conditions_count_dict[full_condition][0], conditions_count_dict[full_condition][1]+1]
            else:
                print(full_condition + ": incorrect condition")
            
    # make a copy of the dictionary -- this will simplify determining which conditions can and can't be compared
    conditions_count_numbs_dict = conditions_count_dict.copy()
    # specifying which pairs should be compared
    for key in conditions_count_dict.keys():
        NHB_value = conditions_count_dict[key][0]
        COPD_value = conditions_count_dict[key][1]
        if NHB_value > 1 and COPD_value > 1: # NHB_value == COPD_value and
            conditions_count_dict[key] = True
        else:
            conditions_count_dict[key] = False
    # initializing box-plot comparisions
    if transwell == "N3":
        box_pairs=[(("control", "NHB"), ("control", "COPD")),
        (("0.01", "NHB"), ("0.01", "COPD")),
        (("0.05", "NHB"), ("0.05", "COPD")),
        (("0.1", "NHB"), ("0.1", "COPD"))
        ]
    else:
        box_pairs=[
        (("0.01", "NHB"), ("0.01", "COPD")),
        (("0.05", "NHB"), ("0.05", "COPD"))
        ]
         
    condition_tracker = {"NHB": 0,
                        "COPD": 1}
    # remove pairs to compare if their corresponding condition does not have at least 2 values
    pairs_to_remove = []
    for pairs in box_pairs:
        MOI = pairs[0][0]
        if MOI == "control":
            rest = ' ' + transwell
        else:
            rest = " MOI " + transwell
        if not conditions_count_dict[MOI+rest]:
            pairs_to_remove.append(pairs)

    for pair_to_remove in pairs_to_remove:
          box_pairs.remove(pair_to_remove)
    
    # custom condition to compare (only compare if data is present, otherwise, error gets thrown)
    condition_pair_to_compare = []
    box_pairs2 = box_pairs
    if condition_to_compare_1=="None" or condition_to_compare_2=="None" or condition_to_compare_1 == condition_to_compare_2:
        box_pairs2 = box_pairs
    else:
        condition_to_compare_1_split = condition_to_compare_1.split(' ')
        condition_to_compare_2_split = condition_to_compare_2.split(' ')
        # stuff for the first condition
        if "control" in condition_to_compare_1:
            MOI_1 = "control"
        else:
            MOI_1 = condition_to_compare_1_split[1]
        condition_1 = condition_to_compare_1_split[0]

        # stuff for the second condition
        if "control" in condition_to_compare_2:
            MOI_2 = "control"
        else:
            MOI_2 = condition_to_compare_2_split[1]
        condition_2 = condition_to_compare_2_split[0]
        
        condition_tracker_1 = condition_tracker[condition_1]
        condition_tracker_2 = condition_tracker[condition_2]
        # check to see if we can even compare these conditions
        try:
            if conditions_count_numbs_dict[" ".join(condition_to_compare_1_split[1:])+" "+transwell][condition_tracker_1] > 1 and conditions_count_numbs_dict[" ".join(condition_to_compare_2_split[1:])+" "+transwell][condition_tracker_2]>1:
                box_pairs2.append(((MOI_1, condition_1), (MOI_2, condition_2)))
        except KeyError:
            nothing=1

    try:
        if box_pairs:
    # conduct statistcal test and annotate plot
            add_stat_annotation(ax, data=cytokine_concs_specify_transwell_df, x='MOI', y=cytokine_name, hue='disease_status',order=conditions_order,
                                box_pairs=box_pairs2,
                                test=statistical_test, text_format=print_p_val, loc='inside', verbose=0)    
    except ValueError:
        if box_pairs:
    # conduct statistcal test and annotate plot
            add_stat_annotation(ax, data=cytokine_concs_specify_transwell_df, x='MOI', y=cytokine_name, hue='disease_status',order=conditions_order,
                                box_pairs=box_pairs,
                                test=statistical_test, text_format=print_p_val, loc='inside', verbose=0) 
    if save_plot:
        output_dir = "../../../figs/cytokine_data_10plex/"
        plt.savefig(output_dir + cytokine_name.replace('/','_') + "_" + transwell+".pdf")
        plt.savefig(output_dir + cytokine_name.replace('/','_') + "_" + transwell+".png")
        plt.savefig(output_dir + cytokine_name.replace('/','_') + "_" + transwell+".svg")
    return(box_pairs)
cytokine_controller()

interactive(children=(Dropdown(description='Cytokine:', options=('ENA-78', 'G-CSF/CSF-3', 'IL-1RA', 'IL-5', 'I…